In [41]:
from __future__ import annotations
import sys; sys.path.insert(0, '..')

%load_ext autoreload
%autoreload 2

# python
import os
import ssl
import csv

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import session_info

from pathlib import Path
from inspect import cleandoc

# utils
from utils import Constants

# stat
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_score # Métrica común para multilabel
from sklearn.exceptions import ConvergenceWarning

# typings
from pandas import DataFrame as PandasDF
from typing import List, Dict, Union

# warnings
import warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore')

# setup
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', None)
# decimals
np.set_printoptions(precision=6)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# cargar el dataset sklearn
if not os.environ.get('CI'):
    ssl._create_default_https_context =\
        ssl._create_unverified_context
          
# rutas absolutas
here: Path = Path.cwd().absolute().parent
data: Path = here / 'data'
poetry_fundation_cleaned: Path = data / 'CleanedPoetryFoundationData.csv'

In [36]:
setup_load:Dict = dict(
    sep=Constants.PIPE_STR,
    quotechar='"',
    quoting=csv.QUOTE_NONNUMERIC,
    encoding=Constants.ENCODING
)

if not poetry_fundation_cleaned.is_file():
    raise FileNotFoundError(
        cleandoc(f'''
        El archivo {poetry_fundation_cleaned} no existe.
        Por favor, descargue el archivo desde:
        https://www.kaggle.com/datasets/abhinavwalia95/poetryfoundationorg
        y coloquelo en la carpeta data.
        ''')
    )
    
poetry_df: PandasDF = (
    pd.read_csv(
        str(poetry_fundation_cleaned), 
        **setup_load
    )
)

poetry_df = poetry_df.loc[~poetry_df.poem.isna(),:]

In [37]:
# Instanciar el binarizador
mlb = MultiLabelBinarizer()

# Ajustar y transformar la columna de etiquetas
# ¡Asegúrate de pasar una lista de listas!
y_mlb = mlb.fit_transform(poetry_df['tags']) 

# y_mlb ahora es una matriz NumPy donde cada fila es una poesía 
# y cada columna es una etiqueta binaria (0 o 1).

# Para ver las etiquetas (el orden de las columnas):
print("Clases (etiquetas únicas):", mlb.classes_)
print("\nMatriz Multilabel (primeras 3 filas):\n", y_mlb[:3])

Clases (etiquetas únicas): [' ' "'" ',' '[' ']' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm'
 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']

Matriz Multilabel (primeras 3 filas):
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0]
 [1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0]
 [1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0]]


In [38]:
# 2. Vectorizar el texto
vectorizer = TfidfVectorizer(max_features=5000) # Limitar a 5000 palabras más importantes
X = vectorizer.fit_transform(poetry_df.poem)

In [42]:
# 1. División de datos (Entrenamiento y Prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_mlb, test_size=0.2, random_state=42
)

# 2. Definición del modelo (Clasificador OVR con Regresión Logística)
classifier = OneVsRestClassifier(
    LogisticRegression(
        solver='liblinear',
        max_iter=1000,  # Aumenta el número de iteraciones
        random_state=42 # Asegura reproducibilidad
    )
)

# 3. Entrenamiento del modelo
classifier.fit(X_train, y_train)

# 4. Predicción
y_pred = classifier.predict(X_test)

# 5. Evaluación
# jaccard_score es una métrica útil para problemas multilabel
jaccard_micro = jaccard_score(y_test, y_pred, average='micro')

print(f"\nJaccard Score (Micro): {jaccard_micro:.4f}")


Jaccard Score (Micro): 0.8164
